Generate student success metric (class label) + Extract key features in the datasets

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [3]:
ac_roster = pd.read_csv("Dataframes/ac_roster.csv").drop(columns=['Unnamed: 0']).drop_duplicates(subset='AndrewID',keep='last')
subjective_measures_of_self_pre = pd.read_csv("Dataframes/subjective_measures_of_self_pre.csv").drop(columns=['Unnamed: 0', 'Timestamp'])
subjective_measures_of_self_post = pd.read_csv("Dataframes/subjective_measures_of_self_post.csv").drop(columns=['Unnamed: 0', 'Timestamp'])
subjective_measures_of_AC = pd.read_csv("Dataframes/subjective_measures_of_AC.csv").drop(columns=['Unnamed: 0', 'Timestamp'])
goals_pre = pd.read_csv("Dataframes/goals_pre.csv").drop(columns=['Unnamed: 0', 'Timestamp']).drop_duplicates(subset='AndrewID')
goals_post = pd.read_csv("Dataframes/goals_post.csv").drop(columns=['Unnamed: 0', 'Timestamp']).drop_duplicates(subset='AndrewID')
demographics = pd.read_csv("Dataframes/demographics.csv").drop(columns=['Unnamed: 0', 'Timestamp'])
initial = pd.read_csv("Dataframes/initial.csv").drop(columns=['Unnamed: 0', 'Timestamp'])

In [4]:
col = "What are some goals you accomplished by attending Academic Coaching sessions? Check all that apply."
col2 = "Was this your first semester working with Academic Coaching?"
goal_cmp = pd.merge(goals_pre, goals_post, on='AndrewID').rename(columns={"Goals" : "Goals_pre", col : "Goals_post"}).drop(
    columns=[col2])
goal_cmp

,AndrewID,Goals_pre,Goals_post
0,priyansj,"Managing your time, Learning to take effective...","Managing your time, Managing stress, Combating..."
1,jianengn,"Managing your time, Learning to take effective...","Managing your time, Preparing for exams, Manag..."
2,mnitke,"Managing your time, Learning to take effective...","Managing your time, Preparing for exams, Manag..."
3,jnnguyen,"Managing your time, Learning to take effective...","Managing your time, Learning to take effective..."
4,akelesog,"Managing your time, Preparing for exams, Manag...","Managing your time, Combating procrastination,..."
...,...,...,...
126,amansuri,"Motivation (reducing procrastination, increasi...",NaN
127,cayalabe,"Managing time (scheduling, etc.), Motivation (...",NaN
128,peilel,"Managing time (scheduling, etc.), Forming prod...",NaN
129,jding2,"Managing time (scheduling, etc.), Motivation (...",NaN


In [5]:
self_cmp = pd.DataFrame(subjective_measures_of_self_post.iloc[:,0]).drop_duplicates(subset='AndrewID')
self_cmp = pd.concat([self_cmp, subjective_measures_of_self_post.iloc[:,1:] - subjective_measures_of_self_pre.iloc[:,1:]], axis=1)

mindset = pd.concat([self_cmp.iloc[:,0], self_cmp.iloc[:,1:4]], axis=1)
optimism = pd.concat([self_cmp.iloc[:,0], self_cmp.iloc[:,4:12]], axis=1)
self_efficacy = pd.concat([self_cmp.iloc[:,0], self_cmp.iloc[:,12:37]], axis=1)
belongingness = pd.concat([self_cmp.iloc[:,0], self_cmp.iloc[:,37:54]], axis=1)


In [6]:
labels = self_cmp[['AndrewID']]
labels['growth_mindset'] = mindset.mean(axis=1)
labels['confidence'] = optimism.mean(axis=1)
labels['self_efficacy'] = self_efficacy.mean(axis=1)
labels['belonging'] = belongingness.mean(axis=1)
labels = labels.fillna(0)
labels

<ipython-input-6-2bd3c62ff447>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['growth_mindset'] = mindset.mean(axis=1)
<ipython-input-6-2bd3c62ff447>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['confidence'] = optimism.mean(axis=1)
<ipython-input-6-2bd3c62ff447>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

,AndrewID,growth_mindset,confidence,self_efficacy,belonging
0,ksabharw,0.666667,-0.625,-0.56,-0.411765
1,priyansj,-0.666667,-0.250,-0.04,-0.529412
2,smabry,-3.666667,0.875,-0.52,1.235294
3,jnnguyen,-0.333333,0.375,0.64,-0.294118
4,etoropov,-4.666667,4.375,2.04,1.117647
...,...,...,...,...,...
142,amansuri,0.000000,-0.125,0.12,-0.750000
143,jding2,0.500000,0.250,0.44,0.166667
144,minsuel,-1.500000,0.125,-0.20,-0.416667
145,ijehanze,0.000000,0.875,-0.20,1.000000


In [7]:
ac_roster['Race'] = ['Asian','Asian','White','White','Asian','White','White','Black',None,'White','Asian','Asian',
                    'Asian',None,'Asian','White','White','Asian','White','White','White','White','White',None,
                    'Asian','Asian','Hispanic','White','Asian','White','Asian','Asian','White','Asian','White',
                    None,'Asian','Asian','White','Asian','Asian','Asian','Asian','Asian','White','White']
ac_roster['Gender'] = ['Male','Female','Male','Female','Male','Male','Female','Male','Male','Female','Female',
                      'Female','Female','Male','Female','Male','Male','Male','Female','Female','Male','Male','Female',
                      'Female','Female','Female','Female','Male','Female','Female','Male','Female','Male','Female',
                      'Female','Female','Male','Female','Male','Male','Female','Male','Female','Male','Male','Male']

In [8]:
ac_roster = ac_roster.drop(columns=['Year','Language(s)','Other','Pref Name','First Name','Full Name','Countries'])

In [9]:
AC_dem = pd.merge(subjective_measures_of_AC[['AndrewID','AC']], ac_roster, left_on='AC', right_on='AndrewID').rename(
    columns={'AndrewID_x' : 'AndrewID','College' : 'School','Class Rank' : 'Class'})
student_AC_demographics = pd.merge(AC_dem, demographics.rename(columns={'Year at the University' : 'Class'}), on='AndrewID', 
                                   suffixes=('_ac','_st')).merge(
    initial[['AndrewID','Major']],on='AndrewID', suffixes=('_ac','_st')).drop_duplicates(subset='AndrewID')
student_AC_demographics = student_AC_demographics.replace(to_replace = ['Domestic (US)',
                                                                       'Domestic (US citizen; permanent resident; or refugee, asylee, or Jay Treaty status)'],
                                                          value='Domestic')
student_AC_demographics


,AndrewID,AC,AndrewID_y,Major_ac,Class_ac,School_ac,Gender_ac,Race_ac,Class_st,School_st,Gender_st,Citizenship status,Race_st,Are you a varsity college athlete?,Did your parent(s)/legal guardian(s) complete a bachelor’s degree?,Major_st
0,ksabharw,jlefevre,jlefevre,Des. Interaction,Graduate,CFA,Male,White,Senior,MCS,Male,International,Asian,NaN,NaN,Mathematical Sciences
1,lkirsh,jlefevre,jlefevre,Des. Interaction,Graduate,CFA,Male,White,Master's student,CIT,Male,Domestic,White,NaN,NaN,Information Security
5,priyansj,ebippus,ebippus,Pub. Policy,Graduate,HC,Female,White,Master's student,CIT,Female,International,Asian,NaN,NaN,ECE
6,yiyid,ebippus,ebippus,Pub. Policy,Graduate,HC,Female,White,Master's student,SCS,Female,International,Asian,NaN,NaN,Software Engineering
7,kjzhang,ebippus,ebippus,Pub. Policy,Graduate,HC,Female,White,Senior,DC,Female,Domestic,Asian,NaN,NaN,Linguistics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,ryanhuan,youngdok,youngdok,Stats & ML,Senior,DC,Male,Asian,First-year,DC,Male,Domestic,Asian,NaN,NaN,IS
221,slaw1,asaalfel,asaalfel,IS + Creative Writing,Sophomore,DC,Male,White,Junior,CIT,Male,Domestic,Asian,NaN,NaN,ChemE
222,mbenitez,mpullen,mpullen,Pro Writ.,Junior,DC,Female,Hispanic,First-year,MCS,Male,Domestic,Hispanic,NaN,NaN,Math
223,sandyz,yingxueh,yingxueh,IS,Sophomore,DC,Female,Asian,First-year,TSB,Female,Domestic,Asian,NaN,NaN,Business Administration


In [10]:
ac_roster[ac_roster['Gender'] == 'Male'].count()

AndrewID      22
Major         20
Class Rank    18
College       20
Gender        22
Race          20
dtype: int64

Demographic information for student and AC: Major, Class, School, Gender, Race

Student: Citizenship status, college athlete, first generation college student

In [11]:
features = student_AC_demographics.rename(columns={'Are you a varsity college athlete?' : 'Athlete',
                                                  'Did your parent(s)/legal guardian(s) complete a bachelor’s degree?' : 'FirstGen'}
                                         )[['AndrewID', 'AC', 'Citizenship status', 'Athlete', 'FirstGen']]
features['Same_Gender'] = (student_AC_demographics['Gender_ac'] == student_AC_demographics['Gender_st'])
features['Same_Major'] = (student_AC_demographics['Major_ac'] == student_AC_demographics['Major_st'])
features['Same_Class'] = (student_AC_demographics['Class_ac'] == student_AC_demographics['Class_st'])
features['Same_School'] = (student_AC_demographics['School_ac'] == student_AC_demographics['School_st'])
features['Same_Race'] = (student_AC_demographics['Race_ac'] == student_AC_demographics['Race_st'])

features['Citizenship status'] = pd.factorize(features['Citizenship status'])[0]
features['Athlete'] = pd.factorize(features['Athlete'])[0]
features['FirstGen'] = pd.factorize(features['FirstGen'])[0]
features

,AndrewID,AC,Citizenship status,Athlete,FirstGen,Same_Gender,Same_Major,Same_Class,Same_School,Same_Race
0,ksabharw,jlefevre,0,-1,-1,True,False,False,False,False
1,lkirsh,jlefevre,1,-1,-1,True,False,False,False,True
5,priyansj,ebippus,0,-1,-1,True,False,False,False,False
6,yiyid,ebippus,0,-1,-1,True,False,False,False,False
7,kjzhang,ebippus,1,-1,-1,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
220,ryanhuan,youngdok,1,-1,-1,True,False,False,True,True
221,slaw1,asaalfel,1,-1,-1,True,False,False,False,False
222,mbenitez,mpullen,1,-1,-1,False,False,False,False,True
223,sandyz,yingxueh,1,-1,-1,True,False,False,False,True


In [12]:
print(np.mean(features['Same_Gender']))
print(np.mean(features['Same_Major']))
print(np.mean(features['Same_Class']))
print(np.mean(features['Same_School']))
print(np.mean(features['Same_Race']))

0.6976744186046512
0.03875968992248062
0.031007751937984496
0.2868217054263566
0.4108527131782946


In [13]:
final_df = pd.merge(features, labels, on='AndrewID').drop(columns=['AndrewID', 'AC'])
final_df

,Citizenship status,Athlete,FirstGen,Same_Gender,Same_Major,Same_Class,Same_School,Same_Race,growth_mindset,confidence,self_efficacy,belonging
0,0,-1,-1,True,False,False,False,False,0.666667,-0.625,-0.56,-0.411765
1,1,-1,-1,True,False,False,False,True,1.666667,-1.500,-0.28,-1.176471
2,0,-1,-1,True,False,False,False,False,-0.666667,-0.250,-0.04,-0.529412
3,0,-1,-1,True,False,False,False,False,-1.666667,1.125,0.84,0.000000
4,1,-1,-1,True,False,False,False,False,0.666667,-1.625,1.12,-0.294118
...,...,...,...,...,...,...,...,...,...,...,...,...
124,1,-1,-1,True,False,False,True,True,-1.000000,-0.375,0.60,0.500000
125,1,-1,-1,True,False,False,False,False,0.500000,-0.500,-0.20,0.583333
126,1,-1,-1,False,False,False,False,True,0.500000,-0.375,0.08,-0.833333
127,1,-1,-1,True,False,False,False,True,-3.000000,0.500,1.20,2.000000


In [25]:
# 90% confidence interval
def confInt(L):
    mu = L.mean()
    sigma = L.std()
    lb = mu - 1.64*(sigma/np.sqrt(len(L)))
    ub = mu + 1.64*(sigma/np.sqrt(len(L)))
    return (lb,ub)

print(confInt(final_df['growth_mindset']))
print(confInt(final_df['confidence']))
print(confInt(final_df['self_efficacy']))
print(confInt(final_df['belonging']))

(-0.47114810326291734, 0.044791514115630454)
(0.009862903318519911, 0.3719200424179142)
(0.11286991188840462, 0.336122336173611)
(0.06851419252418486, 0.32349067143690335)


Gradient Boosted Regression

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

def runGBR(X):
    y = [final_df['growth_mindset'], final_df['confidence'], final_df['self_efficacy'], final_df['belonging']]

    for i in range(len(y)):
        X_train, X_test, y_train, y_test = train_test_split(X, y[i], test_size=0.1, random_state=100)
        reg = GradientBoostingRegressor()
        reg.fit(X_train, y_train)
        mse = mean_squared_error(y_test, reg.predict(X_test))
        print("\ni = %d" % i)
        print("\nMean value: %f" % np.mean(reg.predict(X_test)))
        print("\nMSE: %f" % mse)
        print("\nFeature Importance:")
        print(reg.feature_importances_)
        
X = final_df.iloc[:,0:8]
runGBR(X)


i = 0

Mean value: -0.337755

MSE: 4.451043

Feature Importance:
[0.09096537 0.04958282 0.03243751 0.277883   0.04696335 0.08740295
 0.28916419 0.12560082]

i = 1

Mean value: 0.226354

MSE: 1.743099

Feature Importance:
[0.34551763 0.01054204 0.0206027  0.17687969 0.05412214 0.04053207
 0.16985663 0.18194709]

i = 2

Mean value: 0.217791

MSE: 0.443859

Feature Importance:
[0.38607332 0.02207441 0.0618227  0.21008722 0.08916901 0.07062115
 0.0821518  0.07800039]

i = 3

Mean value: 0.232262

MSE: 0.551347

Feature Importance:
[0.16881799 0.03971249 0.04216789 0.11843286 0.09311152 0.02410548
 0.21117652 0.30247524]


Feature Selection

In [16]:
X = final_df.iloc[:,[0,3,4,6,7]]
runGBR(X)


i = 0

Mean value: -0.416435

MSE: 4.135781

Feature Importance:
[0.17489044 0.35659041 0.19072833 0.12080192 0.1569889 ]

i = 1

Mean value: 0.206819

MSE: 1.732574

Feature Importance:
[0.35482611 0.1805399  0.05916543 0.17922903 0.22623953]

i = 2

Mean value: 0.247806

MSE: 0.447942

Feature Importance:
[0.29079742 0.15309325 0.11737802 0.3736888  0.06504251]

i = 3

Mean value: 0.244585

MSE: 0.617167

Feature Importance:
[0.26482842 0.15996981 0.11419506 0.11245917 0.34854754]


Elastic Net Regression

In [17]:
from sklearn.linear_model import ElasticNet

def runEN(X):
    y = [final_df['growth_mindset'], final_df['confidence'], final_df['self_efficacy'], final_df['belonging']]

    for i in range(len(y)):
        X_train, X_test, y_train, y_test = train_test_split(X, y[i], test_size=0.1, random_state=100)
        reg = ElasticNet()
        reg.fit(X_train, y_train)
        mse = mean_squared_error(y_test, reg.predict(X_test))
        print("\ni = %d" % i)
        print("\nMean value: %f" % np.mean(reg.predict(X_test)))
        print("\nMSE: %f" % mse)
        
X = final_df.iloc[:,0:8]
runEN(X)


i = 0

Mean value: -0.195402

MSE: 3.989037

i = 1

Mean value: 0.183190

MSE: 1.502143

i = 2

Mean value: 0.241379

MSE: 0.277383

i = 3

Mean value: 0.166413

MSE: 0.581161


Neural Network

In [18]:
from sklearn.neural_network import MLPRegressor

X = final_df.iloc[:,0:8]
y = [final_df['growth_mindset'], final_df['confidence'], final_df['self_efficacy'], final_df['belonging']]

for i in range(len(y)):
    X_train, X_test, y_train, y_test = train_test_split(X, y[i], test_size=0.1, random_state=100)
    regr = MLPRegressor(random_state=1, max_iter=500)
    regr.fit(X_train, y_train)
    mse = mean_squared_error(y_test, reg.predict(X_test))
    print("\ni = %d" % i)
    print(mse)
    print("\ni = %d" % i)
    print("\nMean value: %f" % np.mean(reg.predict(X_test)))
    print("\nMSE: %f" % mse)

NameError: name 'reg' is not defined